This program uses Pyspark SQL to extract customer, branch and card data from the provided json file.

In [36]:
# Import libraries 
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql.types import *
from pyspark.sql.functions import * 
#import matplotlib as mpl
#import matplotlib.pyplot as plt
import requests

# Req-1.1 Data Extraction and Transformation with Python and PySpark

In [9]:
# Application to create Dataframes from source
spark = SparkSession.builder.master('local[1]').appName('CreditCardSystems').getOrCreate() 

# Extract the JSON files branch, credit and customer into a dataframe
df_branch = spark.read.json('cdw_sapp_branch.json')  
df_credit = spark.read.json('cdw_sapp_credit.json') 
df_customer = spark.read.json('cdw_sapp_customer.json')

# Register the DataFrame as a SQL temporary view
df_credit.createOrReplaceTempView("credit")
df_customer.createOrReplaceTempView("customer")
df_branch.createOrReplaceTempView("branch")

# 1. Functional Requirements - Load Credit Card Database (SQL)

<b>Data Extraction and Transformation with Python and PySpark. </b><br>
For “Credit Card System,” create a Python and PySpark SQL program to read/extract the following JSON files according to the specifications found in the mapping document.
1. CDW_SAPP_BRANCH.JSON <br>
2. CDW_SAPP_CREDITCARD.JSON <br>
3. CDW_SAPP_CUSTOMER.JSON <br>
Note: Data Engineers will be required to transform the data based on the requirements found in the Mapping Document.
Hint: [You can use PYSQL “select statement query” or simple Pyspark RDD].

In [ ]:
# Application to create Dataframes from source
spark = SparkSession.builder.master('local[1]').appName('CreditCardSystems').getOrCreate() 

# Extract the JSON files branch, credit and customer into a dataframe
df_branch = spark.read.json('cdw_sapp_branch.json')  
df_credit = spark.read.json('cdw_sapp_credit.json') 
df_customer = spark.read.json('cdw_sapp_customer.json')

# Register the DataFrame as a SQL temporary view
df_credit.createOrReplaceTempView("credit")
df_customer.createOrReplaceTempView("customer")
df_branch.createOrReplaceTempView("branch")

In [ ]:
# Adjust customer table acording to the mapping document 

# Convert first and last name to Title Case and middle name to lower case
df_customer = df_customer.withColumn("FIRST_NAME", initcap(df_customer["FIRST_NAME"]))
df_customer = df_customer.withColumn("MIDDLE_NAME", lower(df_customer.MIDDLE_NAME))
df_customer = df_customer.withColumn("LAST_NAME", initcap(df_customer["LAST_NAME"]))
df_customer.select("FIRST_NAME", "MIDDLE_NAME","LAST_NAME").show(10)

In [ ]:
# Concatenate Apartment no and Street name of customer's Residence with comma as a seperator (Street, Apartment)
df_customer = df_customer.withColumn("FULL_STREET_ADDRESS", concat(df_customer["APT_NO"], lit(",") , df_customer["STREET_NAME"]))
df_customer.select("FULL_STREET_ADDRESS").show(10)

In [ ]:
# Change the format of phone number to XXX-XXXX
df_customer.select("CUST_PHONE").show(10)

In [ ]:
# Grab the first the 3 digits 
df_customer = df_customer.withColumn('PHONE_1', split(df_customer['CUST_PHONE'], "\d{4}$"))
df_customer.select("PHONE_1").show(10)

In [ ]:
# Grab the last four digits 
df_customer = df_customer.withColumn('PHONE_2', split(df_customer['CUST_PHONE'], "^\d{3}"))
df_customer.select("PHONE_2").show(10)

In [ ]:
# Concat the first 3 and last 4 digits with - 
df_customer = df_customer.withColumn("CUST_PHONE_FORMATED", concat(df_customer["PHONE_1"], lit("-") , df_customer["PHONE_2"]))
df_customer.select("CUST_PHONE_FORMATED").show(10)

In [ ]:
udf1 = udf(lambda x,y : x+y,ArrayType(StringType()))
df_customer = df_customer.withColumn("CUST_PHONE_FORMATED",udf1('PHONE_1','PHONE_2'))
df_customer.select("CUST_PHONE_FORMATED").show(10)

In [ ]:
# Adjust branch table acording to the mapping document 
# if zipcode is null then load 000000
#df_customer = df_customer.withColumn("FULL_STREET_ADDRESS", concat)
#df_branch.select("BRANCH_ZIP").show(10)

# Req-1.2 Data loading into Database
Once PySpark reads data from JSON files, and then utilizes Python, PySpark, and Python modules to load data into RDBMS(SQL), perform
the following: <br>
a) Create a Database in SQL(MariaDB), named “creditcard_capstone.” <br>
b) Create a Python and Pyspark Program to load/write the “Credit Card System Data” into RDBMS(creditcard_capstone). <br>
Tables should be created by the following names in RDBMS: <br>
CDW_SAPP_BRANCH <br>
CDW_SAPP_CREDIT_CARD <br>
CDW_SAPP_CUSTOMER <br>

In [ ]:
# Create the table CDW_SAPP_BRANCH 
df_branch.write.format("jdbc") \
.mode("append") \
.option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
.option("dbtable", "creditcard_capstone.CDW_SAPP_BRANCH") \
.option("user", "root") \
.option("password", "a") \
.save()

In [ ]:
# Create the table CDW_SAPP_CREDIT_CARD 
df_credit.write.format("jdbc") \
.mode("append") \
.option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
.option("dbtable", "creditcard_capstone.CDW_SAPP_CREDIT_CARD") \
.option("user", "root") \
.option("password", "a") \
.save()

In [ ]:
# Create the table CDW_SAPP_CUSTOMER 
df_customer.write.format("jdbc") \
.mode("append") \
.option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
.option("dbtable", "creditcard_capstone.CDW_SAPP_CUSTOMER") \
.option("user", "root") \
.option("password", "a") \
.save()

# 2. Functional Requirements - Application Front-End
Once data is loaded into the database, we need a front-end (console) to see/display data. For that, create a console-based Python program to satisfy System Requirements 2 (2.1 and 2.2).

<b> Req-2.1 Transaction Details Module </b><br>

# 2.1.1 Display the transactions made by customers living in a given zip code for a given month and year. Order by day in descending order

In [ ]:
# Input fo rmonth, year and zipcode
Month = 8 # Holds the input value for month     | 8
Year = 2018  # Holds the input value for year   | 2018
Zipcode = 39120 # Holds the input value for zipcode | 39120

# Use cdw_app_credit_card table to get TRANSACTION_VALUE, DAY, MONTH AND YEAR
# Use cdw_app_customer to get CUST_ZIP 

sel = "SELECT customer.CUST_ZIP, credit.DAY, credit.MONTH, credit.YEAR, credit.TRANSACTION_TYPE, credit.TRANSACTION_VALUE"
frm = " FROM credit, customer"
where = " WHERE credit.YEAR = " + str(Year) + " AND credit.MONTH = " + str(Month) + " AND customer.CUST_ZIP = " + str(Zipcode)
ordr = " ORDER BY credit.DAY DESC" # Order by ascending 

sqlCredit = spark.sql(sel + frm + where + ordr)
sqlCredit.show(20)


# 2.1.2 Display the number and total values of transactions for a given type

In [ ]:
# Input for a given transaction type 
transact_type = "Bills"

sel = "SELECT TRANSACTION_ID, TRANSACTION_TYPE, TRANSACTION_VALUE"
frm = " FROM credit"
where = " WHERE TRANSACTION_TYPE = " + "\""+ transact_type + "\""
sqlCredit = spark.sql(sel + frm + where)
sqlCredit.show(20)

# 2.1.3 Display the number and total values of transactions for branches in a given state

In [ ]:
# Register the DataFrame as a SQL temporary view
df_branch.createOrReplaceTempView("branch")

# Input for a given state
state = "TX"

sel = "SELECT branch.BRANCH_STATE, credit.TRANSACTION_ID, credit.TRANSACTION_VALUE"
frm = " FROM branch, credit"
where = " WHERE branch.BRANCH_STATE = " + "\""+ state + "\""
sqlCredit = spark.sql(sel + frm + where)
sqlCredit.show(20)

<b>Req-2.2 Customer Details </b><br>
1) Used to check the existing account details of a customer.<br>
2) Used to modify the existing account details of a customer.<br>
3) Used to generate a monthly bill for a credit card number for a given month and year. <br>
4) Used to display the transactions made by a customer between two dates. Order by year, month, and day in descending order. <br>

# 2.2.1 Check the existing account details of a customer


In [ ]:
# Input customer name
first_name = "Alec"
last_name = "Hooper"

sel = "SELECT APT_NO, CREDIT_CARD_NO, CUST_CITY, CUST_COUNTRY, CUST_EMAIL, CUST_PHONE, CUST_STATE, CUST_ZIP"
frm = " FROM customer"
where = " WHERE FIRST_NAME = " + "\""+ first_name + "\"" + "AND LAST_NAME = " + "\"" + last_name + "\""
sqlCredit = spark.sql(sel + frm + where)
sqlCredit.show(20)


# 2.2.2 Modify the exsiting account details of a customer


In [ ]:
# ALTER DATABASE inventory SET DBPROPERTIES ('Edited-by' = 'John', 'Edit-date' = '01/01/2001');

# 2.2.3 Generate a monthly bill for a credit card number for a given month and year


In [ ]:
# Input for month and year bill
month_bill = 8
year_bill = 2018
card_number = 4210653310061055

sel = "SELECT credit.CREDIT_CARD_NO, credit.YEAR, credit.MONTH, SUM(credit.TRANSACTION_VALUE)"
frm = " FROM credit"
where = " WHERE CREDIT_CARD_NO = " + str(card_number) + " AND MONTH = " + str(month_bill) + " AND YEAR = " + str(year_bill) 
grp_by = "GROUP BY CREDIT_CARD_NO"

sql_bill = spark.sql(sel + frm + where + grp_by)
sql_bill.show(20)



# 2.2.4 Display the transactions made by a customer between two date. Order by year, month, and day in desc. 

In [ ]:
# Input for the two dates
Day_1 = 8
Day_2 = 14

sel = "SELECT YEAR, MONTH, DAY, TRANSACTION_VALUE"
frm = " FROM credit"
where = " WHERE DAY BETWEEN " + str(Day_1) + " AND " + str(Day_2) 
ordr_by = " ORDER BY YEAR, MONTH, DAY DESC"

sql_two_date = spark.sql(sel + frm + where + ordr_by )
sql_two_date.show(20)


# 3 - Functional Requirements - Data analysis and Visualization

After data is loaded into the database, users can make changes from the front end, and they can also view data from the front end. Now, the business analyst team wants to analyze and visualize the data according to the below requirements.


# 3.1 Find and plot which transaction type has a high rate of transactions

In [14]:
# Convert pyspark dataframe into pandas dataframe
df_credit_pd = df_credit.toPandas()
df_credit_pd

df_customer_pd = df_customer.toPandas()
df_customer_pd

df_branch_pd = df_branch.toPandas()
df_branch_pd


,BRANCH_CITY,BRANCH_CODE,BRANCH_NAME,BRANCH_PHONE,BRANCH_STATE,BRANCH_STREET,BRANCH_ZIP,LAST_UPDATED
0,Lakeville,1,Example Bank,1234565276,MN,Bridle Court,55044,2018-04-18T16:51:47.000-04:00
1,Huntley,2,Example Bank,1234618993,IL,Washington Street,60142,2018-04-18T16:51:47.000-04:00
2,SouthRichmondHill,3,Example Bank,1234985926,NY,Warren Street,11419,2018-04-18T16:51:47.000-04:00
3,Middleburg,4,Example Bank,1234663064,FL,Cleveland Street,32068,2018-04-18T16:51:47.000-04:00
4,KingOfPrussia,5,Example Bank,1234849701,PA,14th Street,19406,2018-04-18T16:51:47.000-04:00
...,...,...,...,...,...,...,...,...
110,Baldwin,178,Example Bank,1234612707,NY,8th Street West,11510,2018-04-18T16:51:47.000-04:00
111,NewHaven,180,Example Bank,1234347181,CT,Meadow Street,6511,2018-04-18T16:51:47.000-04:00
112,Quincy,188,Example Bank,1234259065,MA,Lincoln Avenue,2169,2018-04-18T16:51:47.000-04:00
113,Lititz,192,Example Bank,1234937904,PA,Glenwood Drive,17543,2018-04-18T16:51:47.000-04:00


In [24]:
# Bargraph 
# x = transaction type 
# y = transaction values 

df_transact = df_credit_pd[['TRANSACTION_TYPE','TRANSACTION_VALUE']]
df_transact = df_transact.groupby(['TRANSACTION_TYPE'])['TRANSACTION_VALUE'].sum()
'''
df_transact.plot(kind='bar', figsize=(10, 6))

plt.xlabel('Transaction Type') # add to x-label to the plot
plt.ylabel('Number of transactions') # add y-label to the plot
plt.title('Transaction Rate For Every Transaction Type') # add title to the plot

plt.show()
'''
df_customer_pd

,APT_NO,CREDIT_CARD_NO,CUST_CITY,CUST_COUNTRY,CUST_EMAIL,CUST_PHONE,CUST_STATE,CUST_ZIP,FIRST_NAME,LAST_NAME,LAST_UPDATED,MIDDLE_NAME,SSN,STREET_NAME
0,656,4210653310061055,Natchez,United States,AHooper@example.com,1237818,MS,39120,Alec,Hooper,2018-04-21T12:49:02.000-04:00,Wm,123456100,Main Street North
1,829,4210653310102868,Wethersfield,United States,EHolman@example.com,1238933,CT,06109,Etta,Holman,2018-04-21T12:49:02.000-04:00,Brendan,123453023,Redwood Drive
2,683,4210653310116272,Huntley,United States,WDunham@example.com,1243018,IL,60142,Wilber,Dunham,2018-04-21T12:49:02.000-04:00,Ezequiel,123454487,12th Street East
3,253,4210653310195948,NewBerlin,United States,EHardy@example.com,1243215,WI,53151,Eugenio,Hardy,2018-04-21T12:49:02.000-04:00,Trina,123459758,Country Club Road
4,301,4210653310356919,ElPaso,United States,WAyers@example.com,1242074,TX,79930,Wilfred,Ayers,2018-04-21T12:49:02.000-04:00,May,123454431,Madison Street
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
947,882,4210653399559239,SiouxCity,United States,FCastle@example.com,1238344,IA,51106,Freida,Castle,2018-04-21T12:49:02.000-04:00,Jonah,123451589,Main Street South
948,470,4210653399650358,Summerville,United States,FBlock@example.com,1240420,SC,29483,Fern,Block,2018-04-21T12:49:02.000-04:00,Aron,123457088,Glenwood Avenue
949,405,4210653399732638,Duluth,United States,DPruitt@example.com,1236149,GA,30096,Dylan,Pruitt,2018-04-21T12:49:02.000-04:00,Lucas,123453259,Country Lane
950,15,4210653399859149,Rowlett,United States,EBeatty@example.com,1236886,TX,75088,Emery,Beatty,2018-04-21T12:49:02.000-04:00,Susanna,123454047,8th Street



# 3.2 Find and plot which state has a high number of customers

In [28]:
# x = all the states 
# y = number of customerws 
df_state = df_customer_pd[['CUST_STATE', 'SSN']]
df_state = df_state.groupby(['CUST_STATE'])['SSN'].count()
df_state

CUST_STATE
AL     5
AR     5
CA    45
CT    14
FL    64
GA    73
IA    21
IL    58
IN    13
KY    17
MA    33
MD    50
MI    44
MN    28
MS    24
MT     9
NC    37
NJ    53
NY    96
OH    44
PA    72
SC    30
TX    33
VA    39
WA    16
WI    29
Name: SSN, dtype: int64


# 3.3 Find and plot the sum of all transactions for each customer, and which customer has the highest transaction amount.hint(use CUST_SSN)



In [35]:
# x = customer
# y = transactions

df_all_transact = df_credit_pd[['CUST_SSN', 'TRANSACTION_VALUE']]
df_all_transact = df_all_transact.groupby(['CUST_SSN'])['TRANSACTION_VALUE'].sum()
df_all_transact 

CUST_SSN
123451007    2037.98
123451012    4025.69
123451037    2122.20
123451041    1085.14
123451068    2810.54
              ...   
123459940      96.27
123459965    2152.70
123459968     964.21
123459978    3091.78
123459988    4760.29
Name: TRANSACTION_VALUE, Length: 952, dtype: float64

# 4. Functional Requirements - LOAN Application Dataset

# 4.1 Create a Python program to GET (consume) data from the above API endpoint for the loan application dataset

In [37]:
url = 'https://raw.githubusercontent.com/platformps/LoanDataset/main/loan_data.json'
r = requests.get(url)
data = r.json()
data

[{'Application_ID': 'LP001002',
  'Gender': 'Male',
  'Married': 'No',
  'Dependents': '0',
  'Education': 'Graduate',
  'Self_Employed': 'No',
  'Credit_History': 1,
  'Property_Area': 'Urban',
  'Income': 'medium',
  'Application_Status': 'Y'},
 {'Application_ID': 'LP001003',
  'Gender': 'Male',
  'Married': 'Yes',
  'Dependents': '1',
  'Education': 'Graduate',
  'Self_Employed': 'No',
  'Credit_History': 1,
  'Property_Area': 'Rural',
  'Income': 'medium',
  'Application_Status': 'N'},
 {'Application_ID': 'LP001005',
  'Gender': 'Male',
  'Married': 'Yes',
  'Dependents': '0',
  'Education': 'Graduate',
  'Self_Employed': 'Yes',
  'Credit_History': 1,
  'Property_Area': 'Urban',
  'Income': 'low',
  'Application_Status': 'Y'},
 {'Application_ID': 'LP001006',
  'Gender': 'Male',
  'Married': 'Yes',
  'Dependents': '0',
  'Education': 'Not Graduate',
  'Self_Employed': 'No',
  'Credit_History': 1,
  'Property_Area': 'Urban',
  'Income': 'low',
  'Application_Status': 'Y'},
 {'Applica

# 4.2 Find the status code of the above API endpoint


In [38]:
r.status_code

200

# 4.3 Once Python reads data from the API, utilize PySpark to load data into RDBMS(SQL). The table name should be CDW-SAPP_loan_application in the database

In [39]:
# Create the table CDW_SAPP_loan_application
df_branch.write.format("jdbc") \
.mode("append") \
.option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
.option("dbtable", "creditcard_capstone.CDW_SAPP_loan_application") \
.option("user", "root") \
.option("password", "a") \
.save()

# 5 - Functional Requirements - Data Analysis and Visualization for Loan Application

1. Find and plot the percentage of applications approved for self-employed applicants. <br>
2. Find the percentage of rejection for married male applicants. <br>
3. Find and plot the top three months with the largest transaction data.<br>
4. Find and plot which branch processed the highest total dollar value of healthcare transactions.